In [1]:
'''7_2_convolutions_for_images.ipynb
Image data is represented as a two-dimensional grid of pixels, be the image monochromatic or in color. 
Accordingly each pixel corresponds to one or multiple numerical values respectively. 
So far we have ignored this rich structure and treated images as vectors of numbers by flattening them, 
irrespective of the spatial relation between pixels. 
This deeply unsatisfying approach was necessary in order to feed the resulting one-dimensional vectors through a fully connected MLP.
'''


import torch
from torch import nn
from d2l import torch as d2l

#### 空间不变性
我们可以从儿童游戏“寻找沃尔多”（Where's Waldo）中汲取一些启示（这个游戏本身已经激发了许多现实生活中的模仿，如图 7.1.1 所示）。游戏包含许多充满活动的混乱场景。在每个场景中，沃尔多都会出现在某个地方，通常潜伏在一些不太可能的地方。读者的目标是找到他。尽管沃尔多有着特征性的服装，找到他却常常出奇地困难，因为场景中存在大量的干扰。然而，沃尔多的样子并不取决于他位于画面的哪个位置。我们可以用一个沃尔多探测器在图像上扫描，为每个区域分配一个分数，表示该区域包含沃尔多的可能性。事实上，许多目标检测和分割算法都是基于这种方法(Long et al., 2015)。卷积神经网络（CNNs）系统化了空间不变性这一思想，并利用它来学习具有较少参数的有用表示。

在实践中，卷积神经网络通过卷积层实现图像数据的空间不变性。卷积层使用卷积核（也称滤波器）在图像数据上滑动，从而提取局部特征。在处理图像数据时，CNN 不仅减少了参数的数量，还能更有效地提取图像特征。这种用较少参数获得更好特征表示的能力，使得卷积神经网络在图像识别、目标检测和语义分割等任务上取得了卓越的性能。

为了有效地处理空间变换，卷积神经网络还使用了汇聚层（Pooling）来降低输入数据的空间维度，减少计算复杂度。这有助于在保持不变性的同时，对特征图进行逐层抽象，使得网络能够捕捉更复杂和高级的特征表示。

总而言之，通过利用空间不变性以及设计灵活的多层结构，卷积神经网络可以高效地分析图像，检测或定位正在寻找的目标，就像一个寻找沃尔多的侦察员一样。这些技术已经在计算机视觉和深度学习领域取得了重大突破，并帮助实现了更有效和可靠的图像识别算法。

#### 基于这种诉求，卷积神经网络应该做到
We can now make these intuitions more concrete by enumerating a few desiderata to guide our design of a neural network architecture suitable for computer vision:

1. In the earliest layers, our network should respond similarly to the same patch, regardless of where it appears in the image. This principle is called translation invariance (or translation equivariance).

2. The earliest layers of the network should focus on local regions, without regard for the contents of the image in distant regions. This is the locality principle. Eventually, these local representations can be aggregated to make predictions at the whole image level.

3. As we proceed, deeper layers should be able to capture longer-range features of the image, in a way similar to higher level vision in nature.